
## 1. Training and Validation
## 2. Testing
## 3. Mask to nii format


# Patch creation

In [ ]:
!pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import CenterCrop
from google.colab.patches import cv2_imshow
from patchify import patchify

# Directory Creation

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

#Load Data

def load_data(train_path):
    train_x = sorted(glob(os.path.join(train_path,"image","neo","*.png")))
    train_y = sorted(glob(os.path.join(train_path,"gt","neo","*.png")))

    return (train_x, train_y)

#Augment Data

def augment_data(image, gt, save_path, augment=True, format="same"):
    size = (1024, 1024)
    for idx, (x, y) in tqdm(enumerate(zip(image, gt)), total=len(image)):
        
        name = x.split("/")[-1].split(".")[0]
        
        #print(name)
          
        img = cv2.resize(cv2.imread(x, cv2.IMREAD_GRAYSCALE), size)
        gt  = cv2.resize(cv2.imread(y, cv2.IMREAD_GRAYSCALE),size )

        

        if augment == True:
            
            # Original Image Patch Creation
            patches_img = patchify(img,(512,512),step=512) # image patch size = step size means there is no overlap in pathes
            
            #print(patches_img.shape)#(16,16,64,64): total patch created 16x16=256,of size:(64x64)
            #print(patches_img[0].shape)(16,64,64)
            
            for i in range(patches_img.shape[0]):
              for j in range(patches_img.shape[1]):
                
                  all_patch_img = patches_img[i,j,:,:]
                  #cv2_imshow(all_patch_img)
                  aug_image_name = f"{name}_{idx}_{i}_{j}.png"
                  aug_image_path = os.path.join(save_path, "image","neo", aug_image_name)
                  cv2.imwrite(aug_image_path, all_patch_img)
                  #print(f"{idx}- {i}-{j}")
            # Ground Truth Patch Creation

            patches_gt = patchify(gt,(512,512),step=512)
            #print(patches_mask.shape)#(16,16,64,64): total patch created 16x16=256,of size:(64x64)
            #print(patches_mask[0].shape)

            for i in range(patches_gt.shape[0]):
              for j in range(patches_gt.shape[1]):
                
                  all_patch_gt = patches_gt[i,j,:,:]
                  #cv2_imshow(all_patch_img)
                  aug_gt_name = f"{name}_{idx}_{i}_{j}.png"
                  aug_gt_path = os.path.join(save_path, "gt","neo", aug_gt_name)
                  cv2.imwrite(aug_gt_path, all_patch_gt)

        

        #break
        idx+=1

if __name__ == "__main__":

    np.random.seed(42)

    train_data_path = "/content/drive/MyDrive/Farhana/DRAC22_Dataset/DRAC22_Renamed_3folders/"
    
    """ Load Data """
    (train_x, train_y) = load_data(train_data_path)

    print(f"For Training :original image {len(train_x)} - ground truth {len(train_y)}")
    

    """ Create directories to save the augmented data """
    create_dir("/content/drive/MyDrive/Farhana/Aug_drac22_data_03/train/image/neo")
    create_dir("/content/drive/MyDrive/Farhana/Aug_drac22_data_03/train/gt/neo")
   

    """ Data augmentation """
    augment_data(train_x, train_y, "/content/drive/MyDrive/Farhana/Aug_drac22_data_03/train/", augment=True, format="same")
    
    print("Augmentation Successful")


For Training :original image 35 - ground truth 35


100%|██████████| 35/35 [00:19<00:00,  1.84it/s]

Augmentation Successful




#Augmentation

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate, GridDistortion, OpticalDistortion, ElasticTransform


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(train_path):
    train_x = sorted(glob(os.path.join(train_path, "image","neo", "*.png")))
    train_y = sorted(glob(os.path.join(train_path, "gt", "neo", "*.png")))
    return (train_x, train_y)

def augment_data(images, masks, save_path, augment=True, format="same"):
    #size = (512, 512)

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        
        name = x.split("/")[-1].split(".")[0]

   
        x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0, border_mode=cv2.BORDER_CONSTANT)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]
            
            aug = GridDistortion(p=1)
            augmented = aug(image=x, mask=y)
            x4 = augmented["image"]
            y4 = augmented["mask"]
            
            aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5, border_mode=cv2.BORDER_CONSTANT)
            augmented = aug(image=x, mask=y)
            x5 = augmented["image"]
            y5 = augmented["mask"]
              
            aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
            augmented = aug(image=x, mask=y)
            x6 = augmented["image"]
            y6 = augmented["mask"]


            X = [x, x1, x2, x3, x4, x5, x6]  
            Y = [y, y1, y2, y3, y4, y5, y6]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
           # i = cv2.resize(i, size)
            #m = cv2.resize(m, size)

            tmp_image_name = f"{name}_{index}.png"
            tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image","neo", tmp_image_name)
            mask_path = os.path.join(save_path, "gt", "neo",tmp_mask_name)
            #print("image")
            #print(image_path)
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1
        #break
        idx+=1    
if __name__ == "__main__":

    np.random.seed(42)


    train_data_path = "/content/drive/MyDrive/Farhana/Aug_drac22_data_03/train/"
    
    (train_x, train_y) = load_data(train_data_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    

    """ Create directories to save the augmented data """
    create_dir("/content/drive/MyDrive/Farhana/Aug_drac22_data_04/train/image/neo")
    create_dir("/content/drive/MyDrive/Farhana/Aug_drac22_data_04/train/gt/neo")
    
    """ Data augmentation """
    augment_data(train_x, train_y, "/content/drive/MyDrive/Farhana/Aug_drac22_data_04/train/", augment=True, format="same")
    

Train: 140 - 140


100%|██████████| 140/140 [00:31<00:00,  4.48it/s]


### Evaluation Metric for Mean Dice and Mean IoU as per DRAC22 Challenge

In [ ]:
import numpy as np

def get_dice(gt, pred, classId=1):
    if np.sum(gt) == 0:
        return np.nan
    else:
        intersection = np.logical_and(gt == classId, pred == classId)
        dice_eff = (2. * intersection.sum()) / (gt.sum() + pred.sum())
        return dice_eff


def get_IoU(gt, pred, classId=1):
    if np.sum(gt) == 0:
        return np.nan
    else:
        intersection = np.logical_and(gt == classId, pred == classId)
        union = np.logical_or(gt == classId, pred == classId)
        iou = np.sum(intersection) / np.sum(union)
        return iou


def get_mean_IoU_dice(gts_list, preds_list):
    assert len(gts_list) == len(preds_list)
    dice_list = []
    iou_list = []
    for gt_array, pred_array in zip(gts_list, preds_list):
        dice = get_dice(gt_array, pred_array, 1)
        iou = get_IoU(gt_array, pred_array, 1)
        dice_list.append(dice)
        iou_list.append(iou)
    mDice = np.nanmean(dice_list)
    mIoU = np.nanmean(iou_list)
    return mDice, mIoU


if __name__ == "__main__":
    # gts_list is a list of ground truth mask images
    # preds_list is a list of predicted mask images
    gt_list = []
    pred_list = []
    mean_Dice, mean_IoU = get_mean_IoU_dice(gt_list, pred_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: Mean of empty slice


### Loss Function : DiceBCE = Dice + BCE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class FocalLoss(nn.CrossEntropyLoss):
    ''' Focal loss for classification tasks on imbalanced datasets '''

    def __init__(self, gamma, alpha=None, ignore_index=-100, reduction='none'):
        super().__init__(weight=alpha, ignore_index=ignore_index, reduction='none')
        self.reduction = reduction
        self.gamma = gamma

    def forward(self, input_, target):
        cross_entropy = super().forward(input_, target)
        # Temporarily mask out ignore index to '0' for valid gather-indices input.
        # This won't contribute final loss as the cross_entropy contribution
        # for these would be zero.
        target = target * (target != self.ignore_index).long()
        input_prob = torch.gather(F.softmax(input_, 1), 1, target.unsqueeze(1))
        loss = torch.pow(1 - input_prob, self.gamma) * cross_entropy
        return torch.mean(loss) if self.reduction == 'mean' else torch.sum(loss) if self.reduction == 'sum' else loss

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)        
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss 
        return Dice_BCE

#Utils

In [ ]:
#Utils
import os
import time
import random
import numpy as np
import cv2
import torch

""" Seeding the randomness. """
def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

""" Create a directory. """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Calculate the time taken """
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#Making the Data Ready for training (Preprocessing data)

In [ ]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from numpy import asarray


class DriveDataset(Dataset):
    def __init__(self, images_path, mask_path, augmentation=None, classes=None):

        self.images_path = images_path
        self.mask_path = mask_path
        self.n_samples = len(images_path)
        self.augmentation=augmentation
        self.classes=classes
        #print("12156") 
        
    def __getitem__(self, index):

        size = (512,512)

        """ Reading image """
        input_image = Image.open(self.images_path[index])
        input_image = input_image.convert("RGB") # gray image to RGB image
        input_image = asarray(input_image)
        input_image = input_image/255.0
        input_image = cv2.resize(input_image, size)
        preprocess = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),]) #Pilimage to Tensor,Normalization in range [0,1]
        input_image = preprocess(input_image)

        """ Reading ground truth"""
        input_mask = Image.open(self.mask_path[index]) 
        input_mask = asarray(input_mask)
        input_mask = input_mask/255.0      
        input_mask = cv2.resize(input_mask, size)
        preprocess2 = transforms.Compose([transforms.ToTensor(),])
        input_mask = preprocess2(input_mask)
        #plt.imshow(input_mask, cmap='gray')
      
        return input_image, input_mask
        
    def __len__(self):
        
        return self.n_samples

if __name__ == "__main__":
  train_x="/content/drive/MyDrive/DRAC22_Dataset/DRAC22_Renamed_3folders/image/ima/075_1.png"
  train_y="/content/drive/MyDrive/DRAC22_Dataset/DRAC22_Renamed_3folders/gt/ima/075_1.png"
  train_dataset = DriveDataset(train_x, train_y)

#Pretrained Model: DeepLabv3_ResNet50

In [ ]:
import torch
import torch.nn as nn
Model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
Model.classifier[4]=nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1)) # changed this layer from outchannel=21 to 1
import sys
class Identity(nn.Module):
  def __init__(self):
    super(Identity,self).__init__()

  def forward(self,x):
    return x
Model.aux_classifier=Identity() # Will not be effected by Aux_classifier layer

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

##Training & Validation

In [ ]:
import os
import time
from glob import glob
from operator import add, sub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import random
import csv
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torch.nn as nn
import gc
#from pytorchtools import EarlyStopping


# This function lodes the training dataset and divide it into two non-overlapping validation and traning datasets

def get_train_valid_loader(data_dir,
                           batch_size,
                           random_seed,
                           kfolde=10,
                           shuffle=True,
                           show_sample=False,
                           num_workers=1,
                           pin_memory=False,
                           shuffelthevaluditation=1): 

    # load the dataset
    train_x = sorted(glob(os.path.join(data_dir,  "image","neo", "*")))[:882]
    train_y = sorted(glob(os.path.join(data_dir, "gt","neo", "*")))[:882]

    valid_x = sorted(glob(os.path.join(data_dir, "image","neo", "*")))[882:]
    valid_y = sorted(glob(os.path.join(data_dir, "gt","neo", "*")))[882:]
    
    """
    print("train image",len(train_x))
    print("train mask",len(train_y))
    print("valid image",len(valid_x)) 
    print("valid mask",len(valid_y)) 

    """
    train_dataset = DriveDataset(train_x, train_y)
    valid_dataset = DriveDataset(valid_x, valid_y)
   
    dataset=ConcatDataset([train_dataset, valid_dataset])
    
    valid_size=(len(dataset)/kfolde)/(len(dataset))
    num_train = len(dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[:split* (int( shuffelthevaluditation) -1 )] + indices[split*int( shuffelthevaluditation):], indices[split * (int( shuffelthevaluditation) -1 ):split * int( shuffelthevaluditation)]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    data_str = f"Dataset Size:\nTrain sampler: {len(train_sampler)} - Validsampler: {len(valid_sampler)}\n"
    
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    
    valid_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    ) 
    
    return (train_loader, valid_loader)

def tensor_to_binary(y_true, y_pred):
    """Ground Truth: Transfering From GPU to CPU and from tensor to numpy """
    y_true = y_true.detach().cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    """ Prediction: Transfering From GPU to CPU and from tensor to numpy  """

    y_pred = y_pred.detach().cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    return y_true, y_pred

def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0
    gt_list = []
    pred_list = []
   
    model.train()
    for x, y in loader:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)['out']
                     
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += float(loss.item())
        
        y, y_pred=tensor_to_binary(y, y_pred) 
        gt_list.append(y)
        pred_list.append(y_pred)
    
    epoch_loss = epoch_loss/len(loader)
    mean_Dice, mean_IoU = get_mean_IoU_dice(gt_list, pred_list)
    print("Training Complete")
    return epoch_loss, mean_Dice, mean_IoU
        
def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0
    gt_list = []
    pred_list = []
   
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)['out']
            loss = loss_fn(y_pred, y)
            epoch_loss += float(loss.item())
                       
            y, y_pred=tensor_to_binary(y, y_pred) 
            gt_list.append(y)
            pred_list.append(y_pred)

    epoch_loss = epoch_loss/len(loader)
    mean_Dice, mean_IoU = get_mean_IoU_dice(gt_list, pred_list)
    return epoch_loss, mean_Dice, mean_IoU 


if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Directories """
    
    create_dir("/content/drive/MyDrive/Farhana/checkpoint/")
    
    """ Load test dataset """
    data_dir="/content/drive/MyDrive/Farhana/Aug_drac22_data_04/train/"

    #early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    
    """ Hyperparameters """
    num_epochs=50
    batch_size=4
    k=10
    lr = 1e-4
    
    #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_ima0.pth"
    #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_npa0.pth"
    checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo0.pth"

    #heckpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_ima2.pth"
    #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo2.pth"
    #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_npa2.pth"


    #checkpoint_path= "/content/drive/MyDrive/Farhana/Result/checkpoint_ima.pth"
    #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_npa.pth"
    #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo.pth"

    """ Dataset and loader """
    
    train_loader, valid_loader=get_train_valid_loader(data_dir=data_dir,
                                                      batch_size=batch_size,
                                                      random_seed=False,
                                                      kfolde=k,
                                                      shuffle=True,
                                                      show_sample=False,
                                                      num_workers=2,
                                                      pin_memory=False,
                                                      shuffelthevaluditation=1)
    print("train data loader",len(train_loader))
    print("valid data loader",len(valid_loader)) 
    
    
    #loaded_checkpoint=torch.load(checkpoint_path)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Model
    model = model.to(device)
    print('Number of model parameters: {}'.format(
      sum([p.data.nelement() for p in model.parameters()])))  
        
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

    #model.load_state_dict(loaded_checkpoint) #loading the trained model

    loss_fn = DiceBCELoss().to(device)
      
    with open('/content/drive/MyDrive/Farhana/drac22_tarin_val_result/submission/train_details_neo_0.csv','a+') as td:
        training_details=csv.writer(td)
        with open('/content/drive/MyDrive/Farhana/drac22_tarin_val_result/submission/val_details_neo_0.csv','a+') as vd:
            val_details=csv.writer(vd)

            """ Training the model """
            best_valid_loss = float("inf")  
            best_tr_dice = 0.000
            best_tr_iou = 0.000
            best_val_dice = 0.000
            best_val_iou = 0.000   

            # Early stopping
            patience = 5
            trigger_times = 0
            

            for epoch in range(num_epochs):
              start_time = time.time()
              train_loss, tr_mean_dice, tr_mean_iou = train(model, train_loader, optimizer, loss_fn, device)
              valid_loss, val_mean_dice, val_mean_iou = evaluate(model, valid_loader, loss_fn, device)

              """ Saving the model """
              if valid_loss < best_valid_loss:
                  data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
                  print(data_str)
              #else:
                 # trigger_times += 1
                 # print('Trigger Times:', trigger_times)

              best_valid_loss = valid_loss
              torch.save(model.state_dict(), checkpoint_path)

              if tr_mean_dice >  best_tr_dice:
                 best_tr_dice = tr_mean_dice

              if tr_mean_iou >  best_tr_iou:
                 best_tr_iou = tr_mean_iou

              if val_mean_dice > best_val_dice:
                 best_val_dice = val_mean_dice

              if val_mean_iou > best_val_iou:
                 best_val_iou = val_mean_iou

              if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                #break


              end_time = time.time()
              epoch_mins, epoch_secs = epoch_time(start_time, end_time)

              data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
              data_str += f'\tTrain Loss: {train_loss:.3f}\n'
              data_str += f'\tVal. Loss: {valid_loss:.3f}'
              print(data_str)

              print(f"\tTrain_Mean_Dice: {tr_mean_dice:1.4f} \tTrain: Mean IoU: {tr_mean_iou:1.4f} ")
              print(f"\tVal_Mean_Dice: {val_mean_dice:1.4f} \tVal: Mean IoU: {val_mean_iou:1.4f} ") 

              training_details.writerow([str(epoch+1),float(train_loss),float(tr_mean_dice),float(tr_mean_iou)])                    
              val_details.writerow([str(epoch+1),float(valid_loss),float(val_mean_dice),float(val_mean_iou)])                                  
                    
              train_loader, valid_loader = get_train_valid_loader(kfolde=k,shuffelthevaluditation=(epoch%k)+1,random_seed=False,data_dir=data_dir,batch_size=batch_size, shuffle=True)
              
              gc.collect()
              
              torch.cuda.empty_cache()  

    print("\nTraining complete")
    print(f"Best train Dice:{best_tr_dice:1.4f}\t Best train IoU:{best_tr_iou:1.4f}")
    print(f"Best Val Dice:{best_val_dice:1.4f}\tBest Val IoU:{best_val_iou:1.4f}")






    
    

train data loader 221
valid data loader 25
Number of model parameters: 39633729
Training Complete
Valid loss improved from inf to 1.1712. Saving checkpoint: /content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo0.pth
Epoch: 01 | Epoch Time: 2m 27s
	Train Loss: 1.334
	Val. Loss: 1.171
	Train_Mean_Dice: 0.0000 	Train: Mean IoU: 0.0000 
	Val_Mean_Dice: 0.0000 	Val: Mean IoU: 0.0000 
Training Complete
Valid loss improved from 1.1712 to 1.0922. Saving checkpoint: /content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo0.pth
Epoch: 02 | Epoch Time: 1m 52s
	Train Loss: 1.119
	Val. Loss: 1.092
	Train_Mean_Dice: 0.0000 	Train: Mean IoU: 0.0000 
	Val_Mean_Dice: 0.0000 	Val: Mean IoU: 0.0000 
Training Complete
Valid loss improved from 1.0922 to 1.0197. Saving checkpoint: /content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo0.pth
Epoch: 03 | Epoch Time: 1m 52s
	Train Loss: 1.056
	Val. Loss: 1.020
	Train_Mean_Dice: 0.0158 	Train: Mean IoU: 0.0091 
	Val_Mean_Dice: 0.0372 	Val: Mean IoU: 0.0206 

#Test the Model


In [ ]:
from tqdm import tqdm
import cv2
import os
import time
from glob import glob
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def predict_data( test_image_path,save_path):
   
    #print(test_image_path)
    #print(save_path)
    for idx, x in tqdm(enumerate(test_image_path), total=len(test_image_path)):
        
        name = x.split("/")[-1].split(".")[0]
        #print(name)
        
        input_image = Image.open(x)
        input_image = input_image.convert("RGB")
        input_image = asarray(input_image)
        input_image = input_image/255.0
        preprocess  = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        input_image = preprocess(input_image)
        input_image = input_image.to(device)
        input_image = torch.unsqueeze(input_image, 0)
        input_image = input_image.float()
        


        
        with torch.no_grad():
          """ Prediction  """
          pred_y = model(input_image)["out"]
          pred_y = torch.sigmoid(pred_y)
          pred_y = pred_y[0].cpu().numpy()   ##(1,512,512)     
          pred_y = np.squeeze(pred_y, axis=0) ## (512,512)   
          pred_y = pred_y > 0.5
          pred_y = np.array(pred_y, dtype=np.uint8)##tensor to nparray conversion
          

        """ Saving masks """
        tmp_pred_name = f"{name}.png"
        pred_path = os.path.join(save_path, tmp_pred_name)
       # print(F"PRED PATH:{pred_path}")
        cv2.imwrite(pred_path, pred_y)
        #break
        idx+=1


if __name__=="__main__":
  """Seeding"""
  seeding(42)

  """Folders"""
  #to save the prediction mask
  #create_dir("/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/IMA0") 
  #create_dir("/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/NPA0") 
  create_dir("/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/NEO0") 

  """Load Data"""
  test_image_path = sorted(glob("/content/drive/MyDrive/Farhana/DRAC2022_Testing_Set/A. Segmentation/1. Original Images/b. Testing Set/*"))

  #predicted_save_path="/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/IMA0/"
  #predicted_save_path="/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/NPA0/" 
  predicted_save_path="/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/NEO0/"

  """ Hyperparameters """
  #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_ima0.pth"
  #checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_npa0.pth"
  checkpoint_path= "/content/drive/MyDrive/Farhana/checkpoint/checkpoint_neo0.pth"
  
  """ Load the checkpoint """
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  model = Model # IMA
  
  model = model.to(device)

  #Loading the save model, Code:#model.load_state_dict(torch.load(PATH)),#model.eval()

  model.load_state_dict(torch.load(checkpoint_path, map_location=device))
 
  model.eval()
  
  predict_data( test_image_path, predicted_save_path)
 

  print("\nPrediction Completed")



100%|██████████| 65/65 [00:08<00:00,  8.02it/s]


Prediction Completed


#Mask to nii

In [ ]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 1.5 MB/s 


In [ ]:
import os
import cv2
import SimpleITK
import numpy as np


def read_nii(nii_path, data_type=np.uint16):
    img = SimpleITK.ReadImage(nii_path)
    data = SimpleITK.GetArrayFromImage(img)
    return np.array(data, dtype=data_type)


def arr2nii(data, filename, reference_name=None):
    img = SimpleITK.GetImageFromArray(data)
    if (reference_name is not None):
        img_ref = SimpleITK.ReadImage(reference_name)
        img.CopyInformation(img_ref)
    SimpleITK.WriteImage(img, filename)


def masks2nii(mask_path):
    mask_name_list = os.listdir(mask_path)
    mask_name_list = sorted(mask_name_list, reverse=False, key=lambda x: int(x[:-4]))
    mask_list = []
    for mask_name in mask_name_list:
        mask = cv2.imread(os.path.join(mask_path, mask_name), -1)
        mask_list.append(mask)
    arr2nii(np.array(mask_list, np.uint8), "3.nii.gz")


if __name__ == "__main__":
    path = "/content/drive/MyDrive/Farhana/DRAC2022_Prediction2/NEO0"
    masks2nii(path)








    
